In [ ]:
""""Thing to be done:
1 correlation
2 correlation with other pages
3 predictive model - neural network


1 compute another graph"""

# Near Real-Time Flu Estimation via Wikipedia

## Introduction



In the following notebook we are going to reproduce for Italy McIver paper et al. "Wikipedia usage estimates Prevalence of Influenza-like illness in theUnited States in near real time". In this paper we show a method of estimating, in near-real time, the level of influenza-like illness (ILI) in Italy by monitoring the rate of particular Wikipedia article views on a daily basis. We calculated on a weekly base the number of times certain influenza- or health-related Wikipedia articles were accessed and compared these data to one of the Italian health protection agency program called "Influnet".

The Notebook in the those three following sections:

* **Comparing Influnet and Wikipedia's Influenza click throught rate**
    * Retrivial and cleaning of the wikipedia pages:
        * Using 3rd party toolkit wikishark
        * Downloading the raw data from https://dumps.wikimedia.org/ (bonus point)
    * Plot the two scaled curves on the same figure
    * Compute Correlation
* **Comparing Influnet data with other Wikipedia's related pages:**
    * Retrivial and Cleaning
    * Plotting and correlation analysis among each other
* **Estimate Flu outbreaks:**
    * Lasso
    * Jesus
    
In the whole notebook we will make use of the following convetions:

## 1 - Comparison between Influnet and  Wikipedia Influenza 

In this section we are going to define the auxiliary functions that I defined in order to perform in order to perform data analysis in a more efficient way: 

In [1]:
import pandas as pd
import numpy as np
from os import listdir
import matplotlib.pyplot as plt
from scipy.stats import pearsonr 
import seaborn as sb
import datetime as dt

In [2]:
def importInflunet(path):
    '''
    Reads the Influnet data and creates a unique multiindex dataframe of the format
    
    (year,week) - incidence
    
    :param path: location of the influnet folder
    :return: compacted version of 
    '''
    parser = lambda d: dt.datetime.strptime(d + '-2', "%Y-%W-%w")
    
    df = pd.concat([pd.read_csv(path+t,
            names=["time","incidence"], sep=" ", parse_dates = ["time"], date_parser = parser, 
        header=1, usecols=[0,4], decimal=",") for t in listdir(path)])
    
    
    
    df = df.set_index(["time"], append=False)
    df["incidence"] = df["incidence"].astype(float)
    df = df.sort_index()
    
    df = df.groupby(df.index).sum()
    alpha = df.index[0]
    omega = df.index[-1]
    time_range = pd.date_range(alpha, omega, freq='W-TUE') #You have to state which is the lasy day of th week
    df = df.reindex(time_range, fill_value= np.nan)
    return df

influnet_raw = importInflunet("/home/aalto/Desktop/DE/hw2/influnet/data/")
print influnet_raw

            incidence
2003-10-28       0.47
2003-11-04       0.52
2003-11-11       0.60
2003-11-18       0.59
2003-11-25       0.70
2003-12-02       0.64
2003-12-09       0.80
2003-12-16       0.99
2003-12-23       1.30
2003-12-30       1.32
2004-01-06       1.71
2004-01-13       2.17
2004-01-20       2.82
2004-01-27       3.67
2004-02-03       4.40
2004-02-10       4.83
2004-02-17       4.75
2004-02-24       4.62
2004-03-02       4.00
2004-03-09       3.56
2004-03-16       3.29
2004-03-23       2.74
2004-03-30       2.15
2004-04-06       1.68
2004-04-13       1.20
2004-04-20       0.77
2004-04-27       0.60
2004-05-04        NaN
2004-05-11        NaN
2004-05-18        NaN
...               ...
2015-09-29        NaN
2015-10-06        NaN
2015-10-13        NaN
2015-10-20        NaN
2015-10-27       0.50
2015-11-03       0.68
2015-11-10       0.76
2015-11-17       0.88
2015-11-24       1.02
2015-12-01       1.16
2015-12-08       1.21
2015-12-15       1.45
2015-12-22       1.66
2015-12-29

In [3]:
def getWikiRaw(wikiPages, path = "/home/aalto/PycharmProjects/digitalepidemiology/data/"):
    '''
    lplp
    :param wikiPages: list of the wikipages that we want to analyze
    :param path: location of the downloaded wikipedia pages
    :return: 
    '''
    df = pd.DataFrame()
    for wikiPage in wikiPages:
        wiki = pd.read_csv(path+wikiPage+".csv", usecols=[0,1], parse_dates=True, index_col=[0], header=None)
        wiki = wiki.resample("W").sum()
        df = df.reindex(wiki.index)
        df[wikiPage] = wiki
    return df

In [4]:
def comparePlots(elems, start):
    for elem in elems:
        y = elem.ix[elem.index.year > start, 0]
        y = y/max(y)
        plt.plot(y)
    plt.show()

In [5]:
influnet = importInflunet("/home/aalto/Desktop/DE/hw2/influnet/data/")
wiki = getWiki(["influenza2"])

comparePlots([influnet, wiki], 2010)

NameError: name 'getWiki' is not defined

In [6]:
getCrossCorrelation([influnet, wiki], range(2010,2017))

NameError: name 'getCrossCorrelation' is not defined

At this point we need the wikipedia pages views count in order to compare them with influnet. In order to satisy the requirement for the bonus point I wrote a simple script that downloads the files from https://dumps.wikimedia.org/, scan it to find the words we are intrested in and writes those entries of the files that we are intrested and write them on different files. 

Once those files are collected and stored on our disk we can call this function which loads the element passed them in memoery and group them by week.


Now to check the previous point we load the influnet dataset in memory and than show its plot

# 2 - Other functions



At this point we are intrested to find if there is any other wikipedia pages that are able to gives us a better insghit about the topics

In [7]:
def getWikis(path = "/home/aalto/PycharmProjects/digitalepidemiology/data/influenza1.csv"):
    '''
    lplp
    :param wikiPages: list of the wikipages that we want to analyze
    :param path: location of the downloaded wikipedia pages
    :return: 
    '''
    df = pd.read_csv(path, parse_dates=['Date'])
    df = df.set_index(["Date"], append=False)
    del df["Week Number "]
    new_columns = dict((column,column[:-4].lower()) for column in df.columns.values)
    df = df.rename(columns = new_columns )
    del df["unname"]
    return df
    

In [8]:
def getCrossCorrelation(elems, years):
    '''
    :param elems: 
    :param years: 
    :return: 
    '''
    y = len(years)
    heatmap = pd.DataFrame(np.zeros(y**2).reshape(y,y), index = years, columns=years)
    
    for y1 in years:
        for y2 in years:
            #print("ciao",elems[0][elems[0].index.year == year1])
            a = elems[0][elems[0].index.year == y1]
            a = a["incidence"]/max(a["incidence"])
            b = elems[1][elems[1].index.year == y2]
            b = b/max(b)
            minimum = min(len(a), len(b))
            heatmap.ix[y1,y2] = pearsonr(a[:minimum].values, b[:minimum].values)[0]
    print heatmap
    sb.heatmap(heatmap)
    plt.show()

In [44]:
influnet_raw = importInflunet("/home/aalto/Desktop/DE/hw2/influnet/data/")
influnet_raw = influnet_raw[(influnet_raw.index.year > 2007) & (influnet_raw.index.year < 2016)]


wikis = getWikis()
wikis = wikis[(wikis.index.year > 2007) & (wikis.index.year < 2016)]


keep =  influnet_raw["incidence"].notnull().values
influnet = influnet_raw[keep]
wikis = wikis[keep]



#for wiki in wikis:
    #plt.plot(influnet/max(influnet["incidence"]))
    #plt.plot(wikis[wiki]/max(wikis[wiki]))
    #plt.show()

In [ ]:
wikis = getWikis()
np.array(wikis)

## 3 - Section

In this section we are going to focus on flu prediction. To estimate flu seasonal outbreaks we used several regression models, that we trained using Wikipedia's pagegivews. We begin our analysis by starting with simple linear model as:

* Linear Regression
* Lasso 
* Elastic Net Regressor

We than proceed in our analysis by tring different non linear regressors. For every model we are going to perform parameters selection to understand which parameters are more relevant in order to undestand and cross-validation to find the best parameters. The models are:

* SVM
* Deep Neural Networks

After this first phase we are going to add to more parameters to our analys:

* Influnet incidence for the week precdiing the target week
* pageviews counts for all the pages that you selected for the week preceeding the target week

and check how good performance increases.


### 3.1 Linear Models

Now we are going to use lasso. After all a Lasso is nothing else than a Elastic net with 

The Lasso is a linear model that estimates sparse coefficients. It is useful in some contexts due to its tendency to prefer solutions with fewer parameter values, effectively reducing the number of variables upon which the given solution is dependent. For this reason, the Lasso and its variants are fundamental to the field of compressed sensing. Under certain conditions, it can recover the exact set of non-zero weights (see Compressive sensing: tomography reconstruction with L1 prior (Lasso)).
Mathematically, it consists of a linear model trained with \ell_1 prior as regularizer. The objective function to minimize is:

$\underset{w}{min\,} { \frac{1}{2n_{samples}} ||X w - y||_2 ^ 2 + \alpha ||w||_1}$

The lasso estimate thus solves the minimization of the least-squares penalty with \alpha ||w||_1 added, where \alpha is a constant and ||w||_1 is the \ell_1-norm of the parameter vector.
The implementation in the class Lasso uses coordinate descent as the algorithm to fit the coefficients. See Least Angle Regression for another implementation:

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from math import sqrt

def predictFluOutbreak(y):
    lassoreg = linear_model.Lasso(alpha=0.1)
    X = []
    Y = influnet[influnet.year == y]
    kf = KFold(n_splits=10, shuffle=True)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        clf.fit(X_train, Y_train)
        y_hat = lassoreg.predict(X_test)
        rms = sqrt(mean_squared_error(y_actual, y_predicted))



In [19]:
def PrintPrediction(X_test, Y_test, Y_hat):
    
    alpha = X_test.index[0]
    omega = X_test.index[-1]
    time_range = pd.date_range(alpha, omega, freq='W-TUE')

    Y_hat = Y_hat.reindex(time_range, fill_value= np.nan)
    Y_test = Y_test.reindex(time_range, fill_value= np.nan)


    plt.plot(Y_hat.index, Y_test['incidence'], linestyle='-', marker='o')
    plt.plot(Y_hat.index, Y_hat,linestyle='-', marker='o')
    plt.show()
    

In [26]:
def LassoPrediction(X_train, Y_train, X_test, Y_test):
    CVlassoreg = LassoCV(alphas = [.0001, .5, 10,1000,0.00000000001],eps = 1e-1000, cv=50, fit_intercept=False)
    CVlassoreg.fit(X_train.values, Y_train['incidence'].values)
    Y_hat = pd.DataFrame(CVlassoreg.predict(X_test.values), index= X_test.index)
    print(len(X_test.index),len(Y_hat), len(Y_test['incidence'].values))
    rms = sqrt(mean_squared_error(Y_test['incidence'].values, Y_hat))
    PrintPrediction(X_test, Y_test, Y_hat)
    return rms, CVlassoreg.get_params()


In [32]:
def ElasticNetPrediction(X_train, Y_train, X_test, Y_test):
    CVelasticreg = ElasticNetCV(l1_ratio=0.9, normalize=False,alphas = [.0001, .5, 10],eps = 1e-1000, cv=50, fit_intercept=False)
    CVelasticreg.fit(X_train.values, Y_train['incidence'].values)
    Y_hat = pd.DataFrame(CVelasticreg.predict(X_test.values), index= X_test.index)
    print(len(X_test.index),len(Y_hat), len(Y_test['incidence'].values))
    rms = sqrt(mean_squared_error(Y_test['incidence'].values, Y_hat))
    PrintPrediction(X_test, Y_test, Y_hat)
    return rms, CVelasticreg.get_params()
    

The previous function could have been done with one line 

In [45]:
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error
from math import sqrt

print 
X_train,Y_train = wikis[(wikis.index.year > 2007) & (wikis.index.year < 2015)], influnet[(influnet.index.year > 2007) & (influnet.index.year < 2015)]  
X_test, Y_test  = wikis[(wikis.index.year == 2015)], influnet[(influnet.index.year == 2015)]

LassoPrediction(X_train, Y_train, X_test, Y_test)
ElasticNetPrediction(X_train, Y_train, X_test, Y_test)





(27, 27, 27)
(27, 27, 27)


(3.649599455058159,
 {'alphas': [0.0001, 0.5, 10],
  'copy_X': True,
  'cv': 50,
  'eps': 0.0,
  'fit_intercept': False,
  'l1_ratio': 0.9,
  'max_iter': 1000,
  'n_alphas': 100,
  'n_jobs': 1,
  'normalize': False,
  'positive': False,
  'precompute': 'auto',
  'random_state': None,
  'selection': 'cyclic',
  'tol': 0.0001,
  'verbose': 0})

We are now intrested in building a better estimation for our model, so we will use in this case two more features.

* Incidence for the previous week
* Pages contus 

In [43]:
tWeek = X_test.index[-1]

wikis["incidence"] =  influnet["incidence"]
Y_predict = wikis["incidence"][wikis["incidence"].index.date == tWeek]
wikis["incidence"] = wikis["incidence"].shift(1)

wikis = wikis.drop(wikis.index[0]) #remove first row

X_train,Y_train = wikis[(wikis.index.year > 2008) & (wikis.index.year < 2015)], influnet[(influnet.index.year > 2008) & (influnet.index.year < 2015)]  
X_test, Y_test  = wikis[(wikis.index.year == 2015)], influnet[(influnet.index.year == 2015)]

LassoPrediction(X_train, Y_train, X_test, Y_test)
ElasticNetPrediction(X_train, Y_train, X_test, Y_test)

(27, 27, 27)
(27, 27, 27)


(1.010117646039902,
 {'alphas': [0.0001, 0.5, 10],
  'copy_X': True,
  'cv': 50,
  'eps': 0.0,
  'fit_intercept': False,
  'l1_ratio': 0.9,
  'max_iter': 1000,
  'n_alphas': 100,
  'n_jobs': 1,
  'normalize': False,
  'positive': False,
  'precompute': 'auto',
  'random_state': None,
  'selection': 'cyclic',
  'tol': 0.0001,
  'verbose': 0})

In [ ]:
print(X_train.index, influnet["incidence"].index)

### 3.2 Non Linear Models

### 3.3 Features Enanchements 

# Stuff

In [ ]:
def padInflunet(aux, year):
    '''
    The influnet dataset lacks information about the weeks that do not belog to the flu season (usally, but not necessarly, from week 17 to 40).
    This functions fills the dataset with empty position in order to match the wikipedia format.
    
    :param aux: Influnet dataframe from a specific year
    :param year: year of the previous Influnet dataframe
    :return: padded version of the original dataframe
    '''
    year_weeks = aux.index.values[-1]
    week_range = range(1,year_weeks+1)
    aux = aux.reindex(week_range, fill_value=0)
    aux["year"] = year
    aux["week"] = week_range
    
    aux.set_index(['year', 'week'], append=False, inplace=True)
    return aux


def getInflunet(path = "/home/aalto/Desktop/DE/hw2/influnet/data/"):
    '''
    import and reformat the original Influnet dataset
    
    :param path: 
    :return: clean and padded version of the Influnet dataset
    '''
    
    df = importInflunet();
    previous = None
    for x,y in df.index.values:
        if previous == None:
            df2 = reindexDF(df.loc[x], x)
        elif x != previous:
            df2 = df2.append(reindexDF(df.loc[x], x))
        previous = x
    return df